Check cuda avilabality

In [2]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")

CUDA available: True


# Scraper

Import libraries

In [3]:
import time
import json
import re
import requests
import pandas as pd
import xmltodict
from duckduckgo_search import DDGS
from pprint import pprint

In [4]:
def fetch_and_parse_xml(rss_xml_url:str='https://trends.google.com/trending/rss?geo=', region:str='US'):
    """
    Fetches XML data (Get request) from a given URL and parses it into a dictionary (using xmltodict).
    Args:
        rss_xml_url (str): The URL of the RSS XML feed to fetch and parse. Defaults to realtime rss feed.
        region (str): The region code to append to the URL. Defaults to 'US'.

    Returns:
        dict: A dictionary representation of the parsed XML data.
              Returns None if an error occurs.

    Raises:
        requests.exceptions.RequestException: If an error occurs while fetching the XML data.
        xmltodict.expat.ExpatError: If an error occurs while parsing the XML data.
        Exception: if any other unexpected error occurs.
    """
    try:
        # Fetch the XML data
        response = requests.get(rss_xml_url+region)
        response.raise_for_status()  # Raise an error for bad status codes
        xml_data = response.content
        # Parse the XML data and convert it to a dictionary
        data_dict = xmltodict.parse(xml_data)
        return data_dict
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the XML data: {e}")
    except xmltodict.expat.ExpatError as e:
        print(f"Error parsing the XML data: {e}")
    except Exception as e:
        print(f'Error: {e}')

WE will choose USA as our geographic location because we will be using small LLMs and embedding models, however if you decide to go with flagship models (ex: GPT4-o...) you can go with non english speaking regions (refer to country_code file)

In [ ]:
# URL for the XML data
#not all countries and region have a daily trending searches rrs feed (for example US, GB, FR... have while MA, DZ, TN don't)
rss_xml_url_daily = 'https://trends.google.com/trends/trendingsearches/daily/rss?geo='
rss_xml_url_realtime = 'https://trends.google.com/trending/rss?geo='

# Fetch, parse, and print the XML data
trends_dict = fetch_and_parse_xml()

pprint(trends_dict)

Pytrends is an unofficial, Python-based open source API for obtaining Google Trends data, but it’s not the a reliable way to gather data and it's slow

SerpApi offers several APIs to access Google data, including Google Trends but it's very expensive (startting from 75$ for 5000 searches/mo)

In [6]:
from pytrends.request import TrendReq

# Only need to run this once, the rest of requests will use the same session.
pytrend = TrendReq()

pytrend.realtime_trending_searches(pn='US') 

,title,entityNames
0,"Cristiano Ronaldo, Manchester United F.C., Eri...","[Cristiano Ronaldo, Manchester United F.C., Er..."
1,"Federico Chiesa, Liverpool F.C., Arne Slot, Ju...","[Federico Chiesa, Liverpool F.C., Arne Slot, J..."
2,"Manchester City F.C., Erling Haaland, Brentfor...","[Manchester City F.C., Erling Haaland, Brentfo..."
3,"Jane's Addiction, Dave Navarro, Perry Farrell,...","[Jane's Addiction, Dave Navarro, Perry Farrell..."
4,"Southampton F.C., Premier League, Russell Mart...","[Southampton F.C., Premier League, Russell Mar..."
5,"Villarreal CF, RCD Mallorca, LaLiga","[Villarreal CF, RCD Mallorca, LaLiga]"
6,"West Ham United F.C., Fulham F.C., Premier League","[West Ham United F.C., Fulham F.C., Premier Le..."
7,"Matthijs de Ligt, Manchester United F.C., Neth...","[Matthijs de Ligt, Manchester United F.C., Net..."
8,"Crystal Palace F.C., Leicester City F.C., Prem...","[Crystal Palace F.C., Leicester City F.C., Pre..."
9,"Brighton & Hove Albion F.C., Ipswich Town F.C....","[Brighton & Hove Albion F.C., Ipswich Town F.C..."


create google trends dataframe

In [7]:
def clean_text(text):
    """
    Clean and normalize the input text by removing special characters and non-ASCII characters.#+

    This function performs the following operations:
    1. Replaces HTML single quote code with an actual single quote.
    2. Removes non-English (non-ASCII) characters.
    3. Removes special characters and punctuation, keeping only letters, numbers, and spaces.

    Args:
        text (str): The input text to be cleaned.

    Returns:
        str: The cleaned and normalized text.
    """
    text = text.replace("&#39;", "'")
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

In [8]:
def create_google_dataframes(trends_dict):
    """
    Creates a pandas DataFrame from a dictionary containing Google Trends data.

    Args:
        trends_dict (dict): A dictionary containing the Google Trends data. It should have the following structure:
            - 'rss' (dict): A dictionary containing the RSS feed data.
                - 'channel' (dict): A dictionary containing the channel data.
                    - 'item' (list): A list of dictionaries containing the individual trend data.
                        - 'title' (str): The title of the trend.
                        - 'ht:approx_traffic' (int): The approximate traffic for the trend.
                        - 'pubDate' (str): The publication date of the trend.
                        - 'ht:news_item' (list or dict): A list or dictionary containing the news items for the trend.

        Note: The 'ht:news_item' field can be either a list or a dictionary. If it is a list, each item in the list is a dictionary with the following keys:
            - 'ht:news_item_url' (str): The URL of the news item.
            - 'ht:news_item_title' (str): The title of the news item.

            If it is a dictionary, it has the following keys:
            - 'ht:news_item_url' (str): The URL of the news item.
            - 'ht:news_item_title' (str): The title of the news item.

    Returns:
        google_trends_df (pandas.DataFrame): A DataFrame containing the Google Trends data. It has the following columns:
            - 'trend_kws' (list): The titles of the trends.
            - 'traffic' (list): The approximate traffic for the trends.
            - 'pubDate' (list): The publication dates of the trends.
            - 'url' (list): The URLs of the news items for each trend.
            - 'title' (list): The titles of the news items for each trend.
            - Note: The 'url' and 'title' columns may contain multiple values for each trend if there are multiple news items.

    """
    google_trends_dict = {"trend_kws":[], "traffic":[], "pubDate":[], "url":[], "title":[]}
    for trend in trends_dict['rss']['channel']['item']:
        google_trends_dict['trend_kws'].append(trend['title'])
        google_trends_dict['traffic'].append(trend['ht:approx_traffic'])
        google_trends_dict['pubDate'].append(trend['pubDate'])
        if isinstance(trend['ht:news_item'], list):
            google_trends_dict['url'].append([news_item['ht:news_item_url'] for news_item in trend['ht:news_item']])
            google_trends_dict['title'].append([news_item['ht:news_item_title'] for news_item in trend['ht:news_item']])
        else:
            google_trends_dict['url'].append([trend['ht:news_item']['ht:news_item_url']])
            google_trends_dict['title'].append([trend['ht:news_item']['ht:news_item_title']])

    google_trends_df = pd.DataFrame(google_trends_dict)
    # clean each title
    google_trends_df["title"] = google_trends_df["title"].map(lambda links: [clean_text(link) for link in links])
    return google_trends_df

In [9]:
google_trends_df = create_google_dataframes(trends_dict)

In [10]:
# remove urls that contain domains from domains_skip_list
domains_skip_list = ["msn.com", "nytimes.com", "washingtonpost.com", "e360.yale.edu", "star-telegram.com"]
for i in range(google_trends_df.shape[0]):
    for j, url in enumerate(google_trends_df.loc[i,"url"]):
        if any(domain in url for domain in domains_skip_list):
            google_trends_df.loc[i,"url"].pop(j)
            google_trends_df.loc[i,"title"].pop(j)

In [11]:
google_trends_df.head()

,trend_kws,traffic,pubDate,url,title
0,missouri football,2000+,"Sat, 14 Sep 2024 06:50:00 -0700",[https://www.espn.com/espn/betting/story/_/id/...,[Boston College vs Missouri betting Tigers off...
1,utsa,200+,"Sat, 14 Sep 2024 06:50:00 -0700",[https://www.statesman.com/story/sports/colleg...,[What channel is Texas vs UTSA on Time TV sche...
2,sicario,200+,"Sat, 14 Sep 2024 06:50:00 -0700",[https://www.eluniversal.com.co/sucesos/2024/0...,[Darmile Peralta el mototaxista que sicarios m...
3,oregon vs oregon state,200+,"Sat, 14 Sep 2024 06:50:00 -0700",[https://www.rollingstone.com/product-recommen...,[Oregon vs Oregon State Livestream How to Watc...
4,lioness movie,200+,"Sat, 14 Sep 2024 06:50:00 -0700",[https://deadline.com/2024/09/lioness-season-2...,[Lioness Adds Max Martini Kirk Acevedo Patric...


related news articles from ddg web search

In [12]:
def create_ddg_dataframe(google_trends_df):
    """
    Creates a DataFrame of DDG news results for each trend keyword in the given Google trends DataFrame.

    Args:
        google_trends_df (pandas.DataFrame): The Google trends DataFrame containing the trend keywords.

    Returns:
        pandas.DataFrame: The DataFrame of DDG news results, with each row representing a news result.
    """

    trends_news = []
    for trend_kw in google_trends_df.trend_kws.to_list():
        # searches for news article with the given keyword, using worldwide region, moderate safe search, and a maximum of 5 results.
        results = DDGS().news(keywords=trend_kw, max_results=5)
        filtered_results = [res for res in results if not any(domain in res['url'] for domain in domains_skip_list)]
        filtered_results = list(map(lambda d: {'trend_kws':trend_kw, **d}, filtered_results[:3]))
        trends_news.extend(filtered_results)
    
    trends_ddg_news_df = pd.DataFrame(trends_news)
    return trends_ddg_news_df

In [ ]:
trends_ddg_news_df = create_ddg_dataframe(google_trends_df)
trends_ddg_news_df.head()

In [14]:
def update_google_trends_with_ddg_news(google_trends_df, trends_ddg_news_df):
    """
    Update the Google Trends DataFrame with additional URLs and titles from DuckDuckGo news search results.

    This function iterates through each trend in the Google Trends DataFrame and finds corresponding
    news articles from the DuckDuckGo news search results. It then appends these additional URLs and
    titles to the existing lists in the Google Trends DataFrame.

    Args:
        google_trends_df (pandas.DataFrame): DataFrame containing Google Trends data.
            Expected to have columns 'trend_kws', 'url', and 'title'.
        trends_ddg_news_df (pandas.DataFrame): DataFrame containing DuckDuckGo news search results.
            Expected to have columns 'trend_kws', 'url', and 'title'.

    Returns:
        None. The function modifies the google_trends_df in-place.
    """
    for i, trend in enumerate(google_trends_df.trend_kws):
        url_list = trends_ddg_news_df[trends_ddg_news_df["trend_kws"]==trend]['url'].to_list()
        title_list = trends_ddg_news_df[trends_ddg_news_df["trend_kws"]==trend]['title'].to_list()
        google_trends_df.loc[i, "url"].extend(url_list)
        google_trends_df.loc[i, "title"].extend(title_list)

In [15]:
update_google_trends_with_ddg_news(google_trends_df, trends_ddg_news_df)

In [ ]:
google_trends_df

# RAG

In [44]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.runnables import RunnableLambda

In [35]:
import asyncio
import aiohttp
from typing import List, Optional
from langchain_core.documents import Document
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import logging
import os

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

async def _fetch(session: aiohttp.ClientSession, url: str, timeout: int = 3) -> Optional[str]:
    try:
        async with session.get(url, timeout=timeout) as response:
            return await response.text()
    except asyncio.TimeoutError:
        logger.warning(f"Timeout occurred while fetching {url}")
        return None
    except Exception as e:
        logger.error(f"Error fetching {url}: {e}")
        return None


async def _fetch_all(urls: List[str], max_concurrent: int = 100, timeout: int = 3) -> List[Optional[str]]:
    connector = aiohttp.TCPConnector(limit=max_concurrent)
    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = [_fetch(session, url, timeout) for url in urls]
        return await asyncio.gather(*tasks)


def _parse_content(html: Optional[str], url: str) -> Optional[Document]:
    if html is None:
        return None
    soup = BeautifulSoup(html, 'html.parser')
    text = soup.get_text(separator='\n', strip=True)
    metadata = {
        "source": url,
        "title": soup.title.string if soup.title else "No title",
    }
    return Document(page_content=text, metadata=metadata)


async def _url_loader(url_list: List[str], max_concurrent: int = 100, timeout: int = 3) -> List[Document]:
    html_contents = await _fetch_all(url_list, max_concurrent, timeout)
    
    with ThreadPoolExecutor(max_workers=(os.cpu_count() or 4)*5) as executor:
        documents = list(executor.map(_parse_content, html_contents, url_list))
    
    return [doc for doc in documents if doc is not None]

By design asyncio does not allow its event loop to be nested. This presents a practical problem - when in an environment where the event loop is already running it's impossible to run tasks and wait for the result. Trying to do so will give the error RuntimeError - This event loop is already running. The issue pops up in various environments, such as web servers, GUI applications and in Jupyter notebooks. This module patches asyncio to allow nested use of asyncio.run and loop.rununtilcomplete.

In [ ]:
# # import nest_asyncio
# # nest_asyncio.apply()

In [77]:
from tqdm.notebook import tqdm
# define embedding model
hf_api_key = ""
# Initialize embeddings based on type
embedding_type = 'api'
hf_model_name= "sentence-transformers/all-MiniLM-l6-v2"
if embedding_type == 'local':
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    embeddings_model = HuggingFaceEmbeddings(
        model_name=hf_model_name,
        model_kwargs={'device': device}
    )
elif embedding_type == 'api':
    embeddings_model = HuggingFaceInferenceAPIEmbeddings(
        api_key=hf_api_key,
        model_name=hf_model_name
    )
else:
    raise ValueError("embedding_type must be either 'local' or 'api'")


# define llm
llm = ChatGroq(
        api_key="",  # Replace with your actual Groq API key
        model_name="llama-3.1-8b-instant"
    )


# define prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful assistant that specializes in article summarization.
            Your task is to summarize a given text article and generate a title for it.
            If the provided article doesn't contain coherent and meaningful content, just return an empty response.
            """,
        ),
        ("human", "Article: {article}"),
    ]
)


# define output schema for llm
class SummmaryWithTitle(BaseModel):
    '''Article summary and title.'''
    title: str
    summary: str

dict_schema = convert_to_openai_tool(SummmaryWithTitle)
structured_output_llm = llm.with_structured_output(dict_schema)

In [95]:
def _rec_splitter(url_doc_list: List[Document]) -> List[Document]:
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=50,
            add_start_index=True
        )
        return text_splitter.split_documents(url_doc_list)

def _retrieved_docs_parser(ret_doc_list: List[Document]) -> str:
    ret_article = "\n".join([doc.page_content for doc in ret_doc_list])
    ret_article = re.sub(r'\n+', '. ', ret_article)
    sentence_pattern = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<![A-Z][a-z][A-Z]\.)(?<=\.|\?|!|\n)\s*')
    ret_article_sentences = sentence_pattern.split(ret_article)
    ret_article_meaningful_sentences = [s for s in ret_article_sentences if len(s.split()) > 5]
    return '\n'.join(ret_article_meaningful_sentences)
retrieved_docs_parser_runnable = RunnableLambda(_retrieved_docs_parser)

In [57]:
from typing import List, Optional, Dict, Any
faiss_cache: Dict[str, FAISS] = {}

In [ ]:
all_url_docs = []
all_url_list = []
trend_kws = google_trends_df.trend_kws.to_list()
for i, trend_kw in enumerate(trend_kws):    
    print(f"Trend: {trend_kw}  {i+1}/{len(trend_kws)}")
    url_list = google_trends_df[google_trends_df["trend_kws"]==trend_kw]['url'].iloc[0]
    if url_list:
        url_docs = await _url_loader(url_list)
        for doc in url_docs:
            if (not doc.page_content) and (doc.metadata["source"] in trends_ddg_news_df.url.to_list()):
                article_body_index = trends_ddg_news_df['url'].to_list().index(doc.metadata["source"])
                doc.page_content += trends_ddg_news_df['body'][article_body_index]
        all_url_docs.append(url_docs)
        all_url_list.append(url_list)

In [ ]:
embeddings_model = HuggingFaceEmbeddings(model_name=hf_model_name,model_kwargs={'device': 'cuda'})

In [ ]:
%%time
from typing import Dict
faiss_cache: Dict[str, FAISS] = {}
for i in range(len(all_url_docs)):
    cache_key = frozenset(all_url_list[i])
    if cache_key not in faiss_cache:
        splits_docs = _rec_splitter(all_url_docs[i])
        faiss_cache[cache_key]  = FAISS.from_documents(splits_docs, embeddings_model)
    faiss_db = faiss_cache[cache_key]
    faiss_retriever = faiss_db.as_retriever(search_type="similarity", search_kwargs={'k': 5})

In [ ]:
embeddings_model = HuggingFaceEmbeddings(model_name=hf_model_name,model_kwargs={'device': 'cpu'})

In [ ]:
%%time
from typing import Dict
faiss_cache: Dict[str, FAISS] = {}
for i in range(len(all_url_docs)):
    cache_key = frozenset(all_url_list[i])
    if cache_key not in faiss_cache:
        splits_docs = _rec_splitter(all_url_docs[i])
        faiss_cache[cache_key]  = FAISS.from_documents(splits_docs, embeddings_model)
    faiss_db = faiss_cache[cache_key]
    faiss_retriever = faiss_db.as_retriever(search_type="similarity", search_kwargs={'k': 5})

In [ ]:
embeddings_model = HuggingFaceInferenceAPIEmbeddings(api_key=hf_api_key,model_name=hf_model_name)


In [ ]:
%%time
from typing import Dict
faiss_cache: Dict[str, FAISS] = {}
for i in range(len(all_url_docs)):
    cache_key = frozenset(all_url_list[i])
    if cache_key not in faiss_cache:
        splits_docs = _rec_splitter(all_url_docs[i])
        faiss_cache[cache_key]  = FAISS.from_documents(splits_docs, embeddings_model)
    faiss_db = faiss_cache[cache_key]
    faiss_retriever = faiss_db.as_retriever(search_type="similarity", search_kwargs={'k': 5})

In [ ]:
embeddings_model = HuggingFaceEmbeddings(model_name=hf_model_name,model_kwargs={'device': 'cuda'})

In [105]:
# define llm
llm = ChatGroq(
        api_key="",  # Replace with your actual Groq API key
        # model_name="llama-3.1-8b-instant"
        model_name="llama-3.1-70b-versatile"

    )


# define prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful assistant that specializes in article summarization.
            Your task is to summarize a given text article and generate a title for it.
            If the provided article doesn't contain coherent and meaningful content, just return an empty response.
            """,
        ),
        ("human", "Article: {article}"),
    ]
)


# define output schema for llm
class SummmaryWithTitle(BaseModel):
    '''Article summary and title.'''
    title: str
    summary: str

dict_schema = convert_to_openai_tool(SummmaryWithTitle)
structured_output_llm = llm.with_structured_output(dict_schema)

In [106]:
trend_kws = google_trends_df.trend_kws.to_list()
results = {"Trend_kws":[], "Title":[], "Summary":[]}
for i, trend_kw in enumerate(trend_kws):    
    print(f"Trend: {trend_kw}  {i+1}/{len(trend_kws)}")
    df_trend = google_trends_df[google_trends_df["trend_kws"]==trend_kw]
    url_list = df_trend['url'].iloc[0]
    if url_list:
        url_docs = await _url_loader(url_list)
        for doc in url_docs:
            if (not doc.page_content) and (doc.metadata["source"] in trends_ddg_news_df.url.to_list()):
                article_body_index = trends_ddg_news_df['url'].to_list().index(doc.metadata["source"])
                doc.page_content += trends_ddg_news_df['body'][article_body_index]
        print("## Creating FAISS vectorstore")
        cache_key = frozenset(url_list)
        if cache_key not in faiss_cache:
            splits_docs = _rec_splitter(url_docs)
            faiss_cache[cache_key]  = FAISS.from_documents(splits_docs, embeddings_model)
        faiss_db = faiss_cache[cache_key]
        faiss_retriever = faiss_db.as_retriever(search_type="similarity", search_kwargs={'k': 10})
        ret_query = '\n'.join(df_trend['title'].iloc[0])

        logger.info("Performing RAG")
        rag_chain = (
            faiss_retriever
            | {"article": retrieved_docs_parser_runnable}
            | prompt_template
            | structured_output_llm
        )
        rag_results = rag_chain.invoke(ret_query)
        results['Trend_kws'].append(trend_kw)
        results['Title'].append(rag_results['title'])
        results['Summary'].append(rag_results['summary'])

2024-09-14 19:52:00,224 - INFO - Performing RAG


Trend: missouri football  1/10


2024-09-14 19:52:01,336 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-14 19:52:01,336 - INFO - Performing RAG


Trend: utsa  2/10


2024-09-14 19:52:02,353 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-14 19:52:02,356 - INFO - Performing RAG


...

2024-09-14 19:52:07,775 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-14 19:52:07,775 - INFO - Performing RAG


Trend: dylan stewart  9/10


2024-09-14 19:52:09,126 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-14 19:52:09,126 - INFO - Performing RAG


Trend: directv disney  10/10


2024-09-14 19:52:10,160 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [107]:
results

{'Trend_kws': ['missouri football',
  'utsa',
  'sicario',
  'oregon vs oregon state',
  'lioness movie',
  'fortnite',
  'arkansas football schedule',
  'pat mcafee',
  'dylan stewart',
  'directv disney'],
 'Title': ['Boston College vs. Missouri Tigers Prediction',
  'UTSA Roadrunners vs. Texas Longhorns: Preview and Streaming Info',
  "Will Smith Exits Big-Budget Action Movie 'Sugar Bandits'",
  'Oregon vs Oregon State Football Game: How to Watch Online Without Cable',
  'Lioness Adds New Cast Members for Season 2',
  'How to Complete The Illusionist Quests in Fortnite Chapter 5, Season 4',
  'Arkansas vs UAB: TV Channel, Time, and Streaming Info',
  "Pat McAfee Praises Shane Beamer, South Carolina Football Ahead of 'College GameDay'",
  'South Carolina Freshman Dylan Stewart Poised for Stardom',
  'DirecTV and Disney Resolve Dispute, Restore Service for College Football'],
 'Summary': ['The No. 6 Missouri Tigers are favored by 14.5 points against the No. 24 Boston College Eagles in